In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [2]:
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])
], axis=0)

train_user = pd.concat([
    pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=500000)
    for x in glob.glob('./训练集/traindata_user/part*')
], axis=0)


In [3]:
testa_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
], axis=0)

testa_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')

In [4]:
#对训练集中的用户ID和商品ID进行标签编码
user_encode = LabelEncoder()
user_encode.fit(list(train_user['user_id']) + list(train_user['user_id']))

goods_encode = LabelEncoder()
goods_encode.fit(list(train_user['goods_id']) + list(train_goods['goods_id']) + list(train_goods['goods_id']))

LabelEncoder()

In [5]:
#计算测试集用户数据中的用户ID和商品ID在训练集用户数据中出现的比例
np.mean(testa_user['user_id'].isin(train_user['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(1.0, 0.9999751455447659)

In [6]:
#去重
train_user['user_id'].nunique(), train_user['goods_id'].nunique()

(51602, 1531577)

In [7]:
#计算训练集用户数据的描述统计信息，并将结果保留两位小数
train_user.describe().round(2)

,is_clk,is_like,is_addcart,is_order,dt
count,7791816.00,7791816.00,7791816.00,7791816.00,7791816.00
mean,1.06,0.02,0.06,0.01,20230222.85
std,0.28,0.13,0.27,0.09,26.21
min,1.00,0.00,0.00,0.00,20230203.00
25%,1.00,0.00,0.00,0.00,20230209.00
50%,1.00,0.00,0.00,0.00,20230216.00
75%,1.00,0.00,0.00,0.00,20230224.00
max,121.00,22.00,28.00,15.00,20230303.00


In [8]:
#将训练集中的用户数据和商品数据根据商品ID进行合并
train_data = pd.merge(train_user.iloc[:], train_goods.iloc[:], on='goods_id')

In [9]:
#获取合并后的数据集中商品类别和品牌的数量
train_data['cat_id'].nunique(), train_data['brandsn'].nunique()
print(train_data)

                                  user_id                          goods_id  \
0        8da2ec07d8bf9bfe1e849cb7e7f25e5c  f6e4f43d18157cbdcdc653c6e35f01fb   
1        62a800cd3e6884078ac86af31926ca6e  f6e4f43d18157cbdcdc653c6e35f01fb   
2        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
3        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
4        328395043a7cf38a36590a8df12b3b03  f6e4f43d18157cbdcdc653c6e35f01fb   
...                                   ...                               ...   
7791759  9b9c0493e6ee675c32b6c2add2b57cd9  49cb5e43cc1dc19cb5f5241cf06816a5   
7791760  2591ff4a1af75a2dc9e020cdafc5ab05  7fb4a90d3aa54ce13355c8ad9043e2e1   
7791761  18917649fd13d3a5fc37443c248d572e  a7e398eff0f997168bf341235ea4817d   
7791762  a4b0bc37d9cceed3bedc794e8d632d38  d6f2d674b397f89a55ecf34f127d0ad3   
7791763  333cb2e2e4c37293ca79f985a66d72f4  2acf9dbcd66278505f54da19d636825c   

         is_clk  is_like  is_addcart  is_order    e

In [10]:
#筛选出合并后的训练数据中满足以下条件的样本：未下单（is_order为0）且加购物车行为不为0（is_addcart不等于0）
train_agg_feat = train_data.loc[
    (train_data['is_order'] == 0) & (train_data['is_addcart'] != 0)
]
print(train_agg_feat)

                                  user_id                          goods_id  \
19       af640ab2869a652c05b0957d9a811704  81de1226af20e196944862b78d27b0b5   
76       11925e60aab947c3b00eee2190fa2ede  ff76680bd6e705fd37df578957faacba   
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
192      0189f417324cc706b727ae1944967c54  3eba0ec94cedb216bca77a07c6525d2d   
202      91aaaea6427f68f1338c6d3273304af2  a92320167ca0b39bea822c690e08c9d7   
...                                   ...                               ...   
7791652  a9ffeecb593fc6d7500ae1f565fda0f0  e1d97c03deb45d7cada039a08de92291   
7791670  353d9d983c3cd90068c59825622b5204  b3d3b112d865b3e99580b2ad49aca38d   
7791734  d2e569e2d1d8f7ad8d86d03ce8b534a0  5c366d4c5045ac3ff303191ddebaf95b   
7791744  fb2d2559a9e77372ffef7c9edebf87c5  c1b254a486172c86297e7a0af07981db   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [11]:
#筛选出train_agg_feat数据集中用户ID存在于测试集用户ID中的样本
train_agg_feat = train_agg_feat[train_agg_feat['user_id'].isin(testa_user['user_id'])]
print(train_agg_feat)

                                  user_id                          goods_id  \
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
...                                   ...                               ...   
7791220  9366b08f49ba3960eb6c58d41609cbd4  d805d53902739d835e6acac868f3ddbe   
7791271  884cfa576e9bd5e7c79307fdebd12ee4  73edfb067d6b0870214b7e51bfc630f5   
7791327  820b7afbd1f4e81a0fcbeee6d3123525  8805c368d93a74e0bdabe7832e8a9dee   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [12]:
#将train_agg_feat数据集中用户ID存在于测试集用户ID中的样本筛选出来
train_agg_feat = train_agg_feat[train_agg_feat['goods_id'].isin(testa_goods['goods_id'])]
print(train_agg_feat)

                                  user_id                          goods_id  \
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
1164     de4997efdcdfa69819e1b0794815bb41  c46542bcd3fb900fa42d57d03cafb727   
...                                   ...                               ...   
7790636  672be194ab66116720214a875208541f  72a4e269893e6471c9d35f4ee0587251   
7791125  4bee2cd0eb5568b44929635e27ed98ea  0c8b7c2a8bca48ba289cc589aad748e0   
7791212  4019534b6bad8cdada154605bae3738f  009126a6f164ab270cb6c106b6a58028   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [13]:
train_agg_feat = train_data.iloc[:].groupby(['user_id', 'goods_id']).agg({
    'is_clk': ['sum', 'max'],
    'is_like': ['sum', 'max'],
    'is_addcart': ['sum', 'max'],
    'is_order': ['sum', 'max'],
})
print(train_agg_feat)

                                                                  is_clk      \
                                                                     sum max   
user_id                          goods_id                                      
0001d0972cc56e776c676d036f3d1a60 000ee6621e5dc775c8e75f758030297c      2   1   
                                 007738408f51e83d3f56897b6b8ecd06      1   1   
                                 02cb92bc89b6264f405b3f15b7b76c9f      1   1   
                                 044e82d678d8c12be0a08ab0c5bde583      1   1   
                                 062c86a124d95036a0910e895efd0eeb      1   1   
...                                                                  ...  ..   
ffffb2aad8fd839b6f152ae9126a05ff fd6907060fb1e080f5a996347d9d2020      1   1   
                                 fef0e42ef8f326a7691053f4a4de000b      1   1   
                                 fefb7378e47a48cf9ee25ebee5d478ff      1   1   
                                 ff8b7d5

In [14]:
#重置索引和重命名列名，将聚合后的列名更改为可读性更好的名称
train_agg_feat = train_agg_feat.reset_index()
train_agg_feat.columns = [
    'user_id',
    'goods_id',
    'is_clk_sum',
    'is_clk_max',
    'is_like_sum',
    'is_like_max',
    'is_addcart_sum',
    'is_addcart_max',
    'is_order_sum',
    'is_order_max'
 ]
print(train_agg_feat)

                                  user_id                          goods_id  \
0        0001d0972cc56e776c676d036f3d1a60  000ee6621e5dc775c8e75f758030297c   
1        0001d0972cc56e776c676d036f3d1a60  007738408f51e83d3f56897b6b8ecd06   
2        0001d0972cc56e776c676d036f3d1a60  02cb92bc89b6264f405b3f15b7b76c9f   
3        0001d0972cc56e776c676d036f3d1a60  044e82d678d8c12be0a08ab0c5bde583   
4        0001d0972cc56e776c676d036f3d1a60  062c86a124d95036a0910e895efd0eeb   
...                                   ...                               ...   
6809676  ffffb2aad8fd839b6f152ae9126a05ff  fd6907060fb1e080f5a996347d9d2020   
6809677  ffffb2aad8fd839b6f152ae9126a05ff  fef0e42ef8f326a7691053f4a4de000b   
6809678  ffffb2aad8fd839b6f152ae9126a05ff  fefb7378e47a48cf9ee25ebee5d478ff   
6809679  ffffb2aad8fd839b6f152ae9126a05ff  ff8b7d565556bcfced37258d69ce6459   
6809680  ffffb2aad8fd839b6f152ae9126a05ff  ff96f17de1a5630cbb0d2da2ffe8f454   

         is_clk_sum  is_clk_max  is_like_sum  is_li

In [15]:
#对聚合后的数据进行进一步的分组聚合操作，计算每个商品的点击次数总和和最大订单数总和，并筛选出点击次数大于100的商品ID
test_goods_id_agg = train_agg_feat.groupby('goods_id').agg({
    'is_clk_sum': 'sum',
    'is_order_max': 'sum'
})
print(test_goods_id_agg)
test_goods_id_agg = test_goods_id_agg[test_goods_id_agg['is_clk_sum'] > 100]
test_valid_goods = test_goods_id_agg.index
print(test_valid_goods)

                                  is_clk_sum  is_order_max
goods_id                                                  
0000023dda9f904c410105d857c6927e          16             0
000003b10b22b2d27df0a1a4c4505abf           6             0
00000983d7e3042755464d42add9c764           1             0
00000cf019f22a5d08cf6e4d1818abe4           1             0
000027e502af2eeaffdbf672ec90a708           6             0
...                                      ...           ...
ffffd179dfe4e14f5ca51b52e9269cbe           1             0
ffffdf0c9e67f479b8e8b545b81c52c6           7             0
ffffe146c8858f0cef9bcd81d70d8f5b          34             1
ffffe8e1352a28c47e85d49bdf3e0bb5           7             0
ffffff57a7cad7becd372c6aad18612a          41             0

[1531526 rows x 2 columns]
Index(['0002686b20b4da254a4020a0bfb95997', '0015cec2763fadb67ce4ba27623358cf',
       '001676d51ac46afd4e091927b7b969d9', '002147e69a005e54e334910fde92fa2d',
       '00606f4ef5b3de3427f6d91861e2cb56', '009

In [16]:
#从聚合后的数据集中选择有下单行为的样本和少量的无下单行为的样本，合并成一个新的训练数据集train_feat_downsample
train_feat_downsmaple = pd.concat([
    train_agg_feat[train_agg_feat['is_order_max'] !=0],
    train_agg_feat[train_agg_feat['is_order_max'] ==0].sample(int(0.03 * len(train_agg_feat)))
], axis=0)
print(train_feat_downsmaple)
print(train_feat_downsmaple['is_order_max'].max())
print(train_feat_downsmaple['is_addcart_max'].max())

                                  user_id                          goods_id  \
390      000c81287b218ec3e2245a33464992fe  8b5a5db81fc5d86c0e4c9de7b8a96f72   
394      000c81287b218ec3e2245a33464992fe  90224d8c0354a3b6383dcf400b45034f   
670      0015421d3e9341bf4f1674f7dd4aa583  0632f9823ec79a93b4ddb9412fc9dcac   
688      0015421d3e9341bf4f1674f7dd4aa583  1b41d6e717ee097750a14ae2b9b0b306   
697      0015421d3e9341bf4f1674f7dd4aa583  21ddf62d1f2ec037a76f84599e16c1eb   
...                                   ...                               ...   
4020067  98506d83368afbea7c079cb5a128c073  d3a5194421fa95b463cab6c03a622d39   
1970681  4aee9b2e11537d17753ca354b33eee83  6de9460f3fe9ae8f2355902dde65a439   
3819065  90d395cab541d8991c4815a5617c1b44  376cb3d36d4cf448e6290916bbec318f   
3867092  92add707414d79a5a1e42c85a36067f4  72df0c7e4acdaa9c8fdcf4eb2aac0b9b   
3881313  934d7a61a0d0309f6318515bbd2ef944  2f982447956f5e1846defc45fc199900   

         is_clk_sum  is_clk_max  is_like_sum  is_li

In [17]:
#计算采样后训练数据集中无下单行为的比例
1 - train_feat_downsmaple['is_order_max'].mean()

0.7795159078177217

In [18]:
#计算采样后训练数据集中无加购物车行为的比例
1 - train_feat_downsmaple['is_addcart_max'].mean()

0.7573705256244325

In [19]:
#进行编码转换
train_feat_downsmaple['user_id'] = user_encode.transform(train_feat_downsmaple['user_id'])
train_feat_downsmaple['goods_id'] = goods_encode.transform(train_feat_downsmaple['goods_id'])

In [20]:
import paddle
import paddle.nn as nn
from paddle.io import Dataset

class SelfDefinedDataset(Dataset):
    def __init__(self, df, mode = 'train'):
        super(SelfDefinedDataset, self).__init__()
        self.df = df
        self.mode = mode

    def __getitem__(self, idx):
        if self.mode == 'predict':
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx],
                self.df['is_like_max'].iloc[idx],
                self.df['is_addcart_max'].iloc[idx],
            )
        else:
            return (
                self.df['user_id'].iloc[idx],
                self.df['goods_id'].iloc[idx],
                self.df['is_clk_max'].iloc[idx],
                (self.df['is_like_max'].iloc[idx]!= 0).astype('int64'),
                (self.df['is_addcart_max'].iloc[idx] != 0).astype('int64'),
                (self.df['is_order_max'].iloc[idx] != 0).astype('int64')
            )

    def __len__(self):
        return len(self.df)

In [21]:
EMBEDDING_SIZE = 256

# 定义深度学习模型
class RecommenderNet(nn.Layer):
    def __init__(self, num_users, num_goods, embedding_size, numeric_size):
        super(RecommenderNet, self).__init__()
        self.num_users = num_users
        self.num_goods = num_goods
        self.embedding_size = embedding_size

        weight_attr_user = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.user_embedding = nn.Embedding(
            num_users,
            embedding_size,
            weight_attr=weight_attr_user
        )

        weight_attr_movie = paddle.ParamAttr(
            regularizer = paddle.regularizer.L2Decay(1e-6),
            initializer = nn.initializer.KaimingNormal()
            )
        self.goods_embedding = nn.Embedding(
            num_goods,
            embedding_size,
            weight_attr=weight_attr_movie
        )

        self.linear = nn.Linear(
            2 * embedding_size, 1
        )

    def forward(self, data):
        user, goods, feat = data[0], data[1], [data[idx] for idx in [2, 3,4]]
        feat = paddle.stack(feat, 1).astype(paddle.float32)

        user_vector = self.user_embedding(user)
        goods_vector = self.goods_embedding(goods)
        x = paddle.concat([user_vector, goods_vector], 1)
        return self.linear(x)

In [22]:
# 定义模型损失函数、优化器和评价指标
model = RecommenderNet(len(user_encode.classes_), len(goods_encode.classes_), 128, 3)
model

RecommenderNet(
  (user_embedding): Embedding(51602, 128, sparse=False)
  (goods_embedding): Embedding(3465659, 128, sparse=False)
  (linear): Linear(in_features=256, out_features=1, dtype=float32)
)

In [23]:
optimizer = paddle.optimizer.SGD(parameters=model.parameters(), learning_rate=0.01)
loss_fn = nn.BCEWithLogitsLoss()

In [24]:
# 模型训练
Train_Loss = []

for epoch in range(10):
    #将数据集的样本顺序进行洗牌
    train_feat_downsmaple = train_feat_downsmaple.sample(frac=1.0)
    #训练集为数据集的前85% 每个批次中的样本数量为128 在每个迭代中对数据进行随机洗牌
    traindataset = SelfDefinedDataset(train_feat_downsmaple.iloc[:-int(-0.15*len(train_feat_downsmaple))])
    train_loader = paddle.io.DataLoader(traindataset, batch_size = 128, shuffle = True)
    #验证集为数据集的后15% 每个批次中的样本数量为128 在每个迭代中对数据进行随机洗牌
    validdataset = SelfDefinedDataset(train_feat_downsmaple.iloc[-int(-0.15*len(train_feat_downsmaple)):])
    valid_loader = paddle.io.DataLoader(validdataset, batch_size = 128, shuffle = True)
    model.train()
    for i, data in enumerate(train_loader):
        pred = model(data)
        #计算损失值
        loss = loss_fn(data[-1].astype(paddle.float32), pred.flatten(), )
        #将损失值添加到训练损失列表中
        Train_Loss.append(loss.item())
        #根据损失值计算梯度
        loss.backward()
        #根据梯度更新模型参数
        optimizer.step()
        #清除梯度信息，为下一次迭代准备
        optimizer.clear_grad()

    # 模型验证
    Val_acc = []
    model.eval()
    with paddle.no_grad():
        for i, data in enumerate(valid_loader):
            pred = model(data)
            Val_acc.append(((pred.flatten() > 0).astype(paddle.int64) == data[-1]).sum().item())
    print(epoch, np.sum(Val_acc) / len(valid_loader.dataset))

0 0.5035862006272442
1 0.5027544202536248
2 0.5024135266578792
3 0.502954411163129
4 0.5034407526930594
5 0.5035180219080951
6 0.503286214262988
7 0.502954411163129
8 0.5025044316167447
9 0.5038770964956139


In [25]:
test_valid_goods = test_valid_goods[test_valid_goods.isin(test_valid_goods)]

test_feat_agg = []
for user in testa_user['user_id'].values:
    for goods in test_valid_goods:
        test_feat_agg.append([user, goods])

In [26]:
test_feat_agg = pd.DataFrame(test_feat_agg)
test_feat_agg.columns = ['user_id', 'goods_id']

In [27]:
test_feat_agg = pd.merge(test_feat_agg, train_agg_feat, on=['user_id', 'goods_id'])

In [28]:
test_feat_agg['user_id'] = user_encode.transform(test_feat_agg['user_id'])
test_feat_agg['goods_id'] = goods_encode.transform(test_feat_agg['goods_id'])

In [29]:
testdataset = SelfDefinedDataset(test_feat_agg)
test_loader = paddle.io.DataLoader(testdataset, batch_size = 128, shuffle = False)

In [30]:
model.eval()

test_pred = []
with paddle.no_grad():
    for i, data in enumerate(test_loader):
        pred = model(data)
        test_pred += list((paddle.nn.functional.sigmoid(pred) > 0.5).numpy().reshape(-1))

In [31]:
test_feat_agg['pred'] = test_pred
test_feat_agg = test_feat_agg[test_feat_agg['pred']]
test_feat_agg['user_id'] = user_encode.inverse_transform(test_feat_agg['user_id'])
test_feat_agg['goods_id'] = goods_encode.inverse_transform(test_feat_agg['goods_id'])

C:\Users\10731\AppData\Local\Temp\ipykernel_27004\2098467592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_feat_agg['user_id'] = user_encode.inverse_transform(test_feat_agg['user_id'])
C:\Users\10731\AppData\Local\Temp\ipykernel_27004\2098467592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_feat_agg['goods_id'] = goods_encode.inverse_transform(test_feat_agg['goods_id'])


In [33]:
test_feat_agg[['user_id', 'goods_id']].to_csv('u2i.csv', index=None, header=None)

In [34]:
import csv

# 定义要添加的列名
new_columns = ["user_id", "goods_id"]

# 打开原始CSV文件并读取数据
with open("u2i.csv", "r") as file:
    reader = csv.reader(file)
    data = list(reader)

# 在第一行插入新的列名
data[0] = new_columns

# 将更新后的数据写入新的CSV文件
with open("u2i_updated.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("列已成功添加到第一行！")

列已成功添加到第一行！
